In [1]:
import os
import numpy as np
import glob
import sys
import subprocess
import argparse
import torch

/home/rchoudhu/.conda/envs/agentformer/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
tepper_path = "./datasets/tepper/Pedestrian_labels/0_frame.txt"
raw_data = np.genfromtxt(tepper_path, delimiter=',')
seq_name = '0'

In [3]:
# Don't do any scaling yet.
# only pedestrians.
gt = raw_data
frames = gt[:, 0].astype(np.float32).astype(int)
fr_start, fr_end = frames.min(), frames.max()
init_frame = fr_start
num_fr = fr_end + 1 - fr_start
past_frames = 8
fut_frames = 12
frame_skip = 1
min_past_frames = 8
min_future_frames = 12 

In [4]:
def get_history_data(frame_idx):
    data_list = []
    for i in range(past_frames):
        if frame_idx - i < init_frame:
            data = []
        data = gt[gt[:, 0] == (frame_idx - i) * frame_skip]
        data_list.append(data)
    return data_list

In [5]:
def get_future_data(frame_idx):
    data_list = []
    for i in range(1, fut_frames + 1):
        data = gt[gt[:, 0] == frame_idx  + i * frame_skip]
        data_list.append(data)
    return data_list

In [6]:
def get_id(data):
    id = []
    for i in range(data.shape[0]):
        id.append(data[i, 1].copy())
    return id

def get_valid_id(pre_data, fut_data):
    cur_id = get_id(pre_data[0])
    valid_id = []
    for idx in cur_id:
        exist_pre = [(False if isinstance(data, list) else (idx in data[:, 1])) for data in pre_data[:min_past_frames]]
        exist_fut = [(False if isinstance(data, list) else (idx in data[:, 1])) for data in fut_data[:min_future_frames]]
        if np.all(exist_pre) and np.all(exist_fut):
            valid_id.append(idx)
    return valid_id

In [20]:
def get_history_motion(data_list, valid_id):
    motion = []
    mask = []
    for identity in valid_id:
        mask_i = torch.zeros(past_frames)
        past_coords = torch.zeros(past_frames, 3)
        for frame_idx in range(past_frames):
            past_data = data_list[frame_idx]
            if len(past_data) > 0 and identity in past_data[:, 1]:
                # Keep all indices (x, y, z), don't apply traj scale.
                found_data = past_data[past_data[:, 1] == identity].squeeze()
                past_coords[past_frames-1-frame_idx, :] = torch.from_numpy(found_data[[2,3,4]]).float()
                mask_i[past_frames - 1 - frame_idx] = 1.0
            elif frame_idx > 0:
                past_coords[past_frames-1 - frame_idx, :] = past_coords[past_frames - frame_idx, :]
            else:
                raise ValueError('current id missing in the first frame!')
        motion.append(past_coords)
        mask.append(mask_i)
    
    return motion, mask

def get_future_motion(data_list, valid_id):
    motion = []
    mask = []
    for identity in valid_id:
        mask_i = torch.zeros(fut_frames)
        mask_i = torch.zeros(fut_frames)
        pos_3d = torch.zeros([fut_frames, 3])
        for j in range(fut_frames):
            fut_data = data_list[j]              # cur_data
            if len(fut_data) > 0 and identity in fut_data[:, 1]:
                found_data = fut_data[fut_data[:, 1] == identity].squeeze()[[2,3,4]]
                pos_3d[j, :] = torch.from_numpy(found_data).float()
                mask_i[j] = 1.0
            elif j > 0:
                pos_3d[j, :] = pos_3d[j - 1, :]    # if none, copy from previous
            else:
                raise ValueError('current id missing in the first frame!')
        motion.append(pos_3d)
        mask.append(mask_i)
    
    return motion, mask

In [21]:
def get_data(frame_idx):
    assert frame_idx - init_frame >= 0 and frame_idx - init_frame < num_fr

    pre_data = get_history_data(frame_idx)
    fut_data = get_future_data(frame_idx)
    valid_id = get_valid_id(pre_data, fut_data)

    if len(pre_data[0]) == 0 or len(fut_data[0]) == 0 or len(valid_id) == 0:
        return None
    pred_mask = None
    heading = None
    
    pre_motion_3D, pre_motion_mask = get_history_motion(pre_data, valid_id)
    fut_motion_3D, fut_motion_mask = get_future_motion(fut_data, valid_id)

    data = {
        'pre_motion_3D': pre_motion_3D,
        'fut_motion_3D': fut_motion_3D,
        'fut_motion_mask': fut_motion_mask,
        'pre_motion_mask': pre_motion_mask,
        'pre_data': pre_data,
        'fut_data': fut_data,
        'heading': heading,
        'valid_id': valid_id,
        'pred_mask': pred_mask,
        'scene_map': None,
        'seq': seq_name,
        'frame': frame_idx
    }

    return data


In [22]:
# Great, dataloader is deon. Now, how do we include it in?

{'pre_motion_3D': [tensor([[-13.6900,   9.6600,   0.0000],
          [-14.4000,   8.7300,   0.0000],
          [-15.0100,   8.2200,   0.0000],
          [-15.8200,   7.0900,   0.0000],
          [-16.6100,   6.7400,   0.0000],
          [-17.2800,   6.0200,   0.0000],
          [-18.4300,   4.8000,   0.0000],
          [-19.5200,   3.8000,   0.0000]]),
  tensor([[177.7100,  43.5800,   0.0000],
          [177.7900,  43.3000,   0.0000],
          [177.2500,  42.8400,   0.0000],
          [176.6100,  42.1900,   0.0000],
          [176.0800,  41.7300,   0.0000],
          [175.9700,  41.5400,   0.0000],
          [175.9500,  40.8300,   0.0000],
          [175.4100,  40.3700,   0.0000]])],
 'fut_motion_3D': [tensor([[-20.5600,   3.0100,   0.0000],
          [-21.4100,   2.4500,   0.0000],
          [-22.1400,   1.5200,   0.0000],
          [-22.9100,   0.6100,   0.0000],
          [-23.9100,  -0.1200,   0.0000],
          [-24.4100,  -1.1500,   0.0000],
          [-25.5100,  -2.0800,   0.00

In [ ]:
# okay dataloader is essentially done. Next, need to get the thing training with it.
# ugh its so much work